
# Energy Grid Balancing — MIP (PuLP) + QUBO (neal) on One Dataset (Colab-Ready)

This notebook balances supply and demand on a small power grid over multiple time periods using one **synthetic dataset**.

1) **Classical MIP (economic dispatch with DC power flow & storage)**  
   - Variables: generator outputs \(p_{g,t}\), bus voltage angles \(\theta_{n,t}\), line flows \(f_{\ell,t}\), storage charge/discharge/energy \(c_t,d_t,e_t\), and optional load shed \(u_{n,t}\).  
   - Constraints: nodal power balance, generator limits & ramps, line capacities via DC approximation, storage dynamics & bounds, optional curtailment/shed bounds.  
   - Objective: minimize generation cost + small penalties for curtailment/shed.

2) **QUBO (binary on/off per generator & time)**  
   - Variables \(y_{g,t}\in\{0,1\}\).  
   - Penalties: meet **system-wide** net demand (renewables subtract), ramp smoothness \((y_{g,t}-y_{g,t-1})^2\), soft commitment count.  
   - Cost: on-cost proportional to marginal cost × \(P^{\max}\).  
   - Solve with **simulated annealing** (`neal`), then perform a **continuous dispatch** given the committed set.

We compare cost & feasibility and visualize demand vs. supply, storage SOC, and line flows.


In [ ]:

# Install dependencies if needed (works in Colab)
def _silent_imports():
    flags = {"pulp": False, "dimod": False, "neal": False}
    try:
        import pulp
        flags["pulp"] = True
    except Exception:
        pass
    try:
        import dimod
        flags["dimod"] = True
    except Exception:
        pass
    try:
        import neal
        flags["neal"] = True
    except Exception:
        pass
    return flags

flags = _silent_imports()
if not flags["pulp"]:
    %pip -q install pulp
if not flags["dimod"] or not flags["neal"]:
    %pip -q install dimod neal

flags = _silent_imports()
print("PuLP:", flags["pulp"], "| dimod:", flags["dimod"], "| neal:", flags["neal"])


In [ ]:

# ==== One Synthetic Grid Dataset ====
import numpy as np, pandas as pd

rng = np.random.default_rng(2027)

# Grid topology
Nbus = 5
buses = [f"B{i+1}" for i in range(Nbus)]
# Lines: (from,to,B (susceptance), capacity)
lines = [
    ("B1","B2", 1.0, 40.0),
    ("B2","B3", 1.0, 40.0),
    ("B3","B4", 1.0, 35.0),
    ("B4","B5", 1.0, 35.0),
    ("B5","B1", 0.8, 30.0),
    ("B2","B5", 0.6, 25.0),
]

# Generators: (name, bus, pmin, pmax, marginal_cost, ramp)
gens = [
    ("G1","B1",  0.0, 50.0, 18.0, 20.0),
    ("G2","B2",  0.0, 40.0, 22.0, 15.0),
    ("G3","B3",  0.0, 35.0, 30.0, 12.0),
    ("G4","B4",  0.0, 25.0, 45.0, 10.0),
    ("G5","B5",  0.0, 30.0, 28.0, 12.0),
]

# Renewables (zero marginal) at some buses: (bus, type, capacity)
re_new = [
    ("B1","solar", 35.0),
    ("B3","wind",  30.0),
]

T = 12  # time periods
# Demand per bus per time (MW), sinusoidal + noise
base_load = np.array([30, 28, 26, 22, 24])  # per bus baseline
demand = []
for t in range(T):
    scale = 1.0 + 0.35*np.sin(2*np.pi*(t/T))  # daily cycle
    noise = rng.normal(0, 1.5, size=Nbus)
    demand.append(np.maximum(0.0, base_load*scale + noise))
demand = np.vstack(demand)  # T x Nbus

# Renewable profiles
solar_profile = np.clip(np.sin(np.linspace(0, np.pi, T)), 0, None)  # sunrise to sunset
wind_profile = np.clip(0.6 + 0.3*np.sin(np.linspace(0, 4*np.pi, T)+0.7) + rng.normal(0,0.05,T), 0, 1.0)

ren_time_bus = np.zeros((T, Nbus))
for (bus,typ,cap) in re_new:
    b = buses.index(bus)
    prof = solar_profile if typ=="solar" else wind_profile
    ren_time_bus[:, b] += cap * prof

# Storage at B5
E_max = 40.0  # MWh
P_ch_max = 15.0
P_dis_max = 15.0
eta_ch = 0.95
eta_dis = 0.95
e0 = 20.0

# Line incidence helper
line_df = pd.DataFrame(lines, columns=["fr","to","B","Fmax"])
bus_idx = {b:i for i,b in enumerate(buses)}
line_df["i"] = line_df["fr"].map(bus_idx)
line_df["j"] = line_df["to"].map(bus_idx)

# Package dataframes
gen_df = pd.DataFrame(gens, columns=["gen","bus","pmin","pmax","c","ramp"])
dem_df = pd.DataFrame(demand, columns=buses)
ren_df = pd.DataFrame(ren_time_bus, columns=buses)
print("Buses:", buses)
print("Lines:\n", line_df[["fr","to","Fmax"]].to_string(index=False))
print("Generators:\n", gen_df.to_string(index=False))
print("Time periods:", T)



## Part 1 — Classical MIP: DC Power Flow + Storage

**Variables (per time \(t\)):**
- \(p_{g,t}\): generator output  
- \(\theta_{n,t}\): bus phase angle (fix one slack angle per \(t\))  
- \(f_{\ell,t}\): line flow (from→to positive) with \(f_{\ell,t} = B_\ell(\theta_{i,t} - \theta_{j,t})\)  
- Storage at B5: charge \(c_t\), discharge \(d_t\), energy \(e_t\)  
- Optional load shed \(u_{n,t} \ge 0\) with high penalty

**Constraints:**  
- Nodal balance at each bus & time (gen + renew + inflow − outflow − demand − net charge/discharge − shed = 0)  
- Generator bounds & ramping  
- DC line equations & capacities  
- Storage power & energy bounds, dynamics \(e_{t+1} = e_t + \eta_{ch} c_t - d_t/\eta_{dis}\)

**Objective:** \(\min \sum_{g,t} c_g p_{g,t} + \lambda_{shed} \sum_{n,t} u_{n,t}\).


In [ ]:

import numpy as np, pandas as pd

try:
    import pulp
    HAVE_PULP = True
except Exception:
    HAVE_PULP = False

def solve_mip_dc(gen_df, line_df, dem_df, ren_df, buses, T, storage_bus="B5",
                 E_max=40.0, P_ch_max=15.0, P_dis_max=15.0, eta_ch=0.95, eta_dis=0.95, e0=20.0):
    if not HAVE_PULP:
        return "NoPuLP", None

    Nbus = len(buses)
    gens_by_bus = {b: gen_df[gen_df["bus"]==b]["gen"].tolist() for b in buses}
    g_idx = {row["gen"]: i for i,row in gen_df.iterrows()}

    prob = pulp.LpProblem("DC_OPF_Storage", pulp.LpMinimize)

    # Variables
    p = pulp.LpVariable.dicts("p", (gen_df["gen"].tolist(), range(T)), lowBound=0)
    theta = pulp.LpVariable.dicts("theta", (buses, range(T)), lowBound=None)
    f = pulp.LpVariable.dicts("f", (range(len(line_df)), range(T)), lowBound=None)
    u = pulp.LpVariable.dicts("shed", (buses, range(T)), lowBound=0)

    # Storage at storage_bus
    c = pulp.LpVariable.dicts("charge", range(T), lowBound=0, upBound=P_ch_max)
    d = pulp.LpVariable.dicts("discharge", range(T), lowBound=0, upBound=P_dis_max)
    e = pulp.LpVariable.dicts("energy", range(T), lowBound=0, upBound=E_max)

    # Objective
    shed_pen = 500.0
    prob += pulp.lpSum(gen_df.set_index("gen").loc[g,"c"] * p[g][t] for g in gen_df["gen"] for t in range(T)) \
            + shed_pen * pulp.lpSum(u[b][t] for b in buses for t in range(T))

    # Bounds & ramp
    for _, row in gen_df.iterrows():
        g = row["gen"]; pmin, pmax, ramp = row["pmin"], row["pmax"], row["ramp"]
        for t in range(T):
            prob += p[g][t] >= pmin
            prob += p[g][t] <= pmax
        for t in range(1, T):
            prob += p[g][t] - p[g][t-1] <= ramp
            prob += p[g][t-1] - p[g][t] <= ramp

    # DC line equations and limits
    for ell, lrow in line_df.iterrows():
        i = lrow["i"]; j = lrow["j"]; B = lrow["B"]; Fmax = lrow["Fmax"]
        bi = buses[i]; bj = buses[j]
        for t in range(T):
            prob += f[ell][t] == B*(theta[bi][t] - theta[bj][t])
            prob += f[ell][t] <= Fmax
            prob += f[ell][t] >= -Fmax

    # Slack angle fix (set B1 angle = 0 at all times)
    for t in range(T):
        prob += theta["B1"][t] == 0.0

    # Storage dynamics: at B5
    sb = storage_bus
    b_index = buses.index(sb)
    for t in range(T):
        if t == 0:
            prob += e[t] == e0 + eta_ch * c[t] - (1.0/eta_dis) * d[t]
        else:
            prob += e[t] == e[t-1] + eta_ch * c[t] - (1.0/eta_dis) * d[t]

    # Nodal balance
    inc = {b: [] for b in buses}
    out = {b: [] for b in buses}
    for ell, lrow in line_df.iterrows():
        bi = lrow["fr"]; bj = lrow["to"]
        inc[bj].append(ell); out[bi].append(ell)

    for t in range(T):
        for b in buses:
            gen_sum = pulp.lpSum(p[g][t] for g in gens_by_bus[b]) if gens_by_bus[b] else 0.0
            ren = float(ren_df.loc[t, b])
            load = float(dem_df.loc[t, b])
            inflow = pulp.lpSum(f[ell][t] for ell in inc[b]) if inc[b] else 0.0
            outflow = pulp.lpSum(f[ell][t] for ell in out[b]) if out[b] else 0.0
            shed = u[b][t]

            # storage terms only at storage bus
            ch = c[t] if b==sb else 0.0
            dis = d[t] if b==sb else 0.0

            prob += gen_sum + ren + inflow - outflow - load - ch + dis + shed == 0.0

    # Solve
    _ = prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    if status not in ("Optimal","Feasible"):
        return status, None

    # Extract solution
    sol = {
        "p": {(g,t): float(p[g][t].value()) for g in gen_df["gen"] for t in range(T)},
        "theta": {(b,t): float(theta[b][t].value()) for b in buses for t in range(T)},
        "f": {(ell,t): float(f[ell][t].value()) for ell in range(len(line_df)) for t in range(T)},
        "shed": {(b,t): float(u[b][t].value()) for b in buses for t in range(T)},
        "c": {t: float(c[t].value()) for t in range(T)},
        "d": {t: float(d[t].value()) for t in range(T)},
        "e": {t: float(e[t].value()) for t in range(T)},
        "obj": float(pulp.value(prob.objective))
    }
    return status, sol

mip_status, mip_sol = solve_mip_dc(gen_df, line_df, dem_df, ren_df, buses, T,
                                   E_max, P_ch_max, P_dis_max, eta_ch, eta_dis, e0)
print("MIP status:", mip_status)
if mip_sol:
    print("MIP objective cost:", round(mip_sol["obj"],2))


In [ ]:

# MIP summaries & plots
import numpy as np, pandas as pd, matplotlib.pyplot as plt

if mip_sol:
    Pg = pd.DataFrame({g: [mip_sol["p"][(g,t)] for t in range(T)] for g in gen_df["gen"]})
    Shed = pd.DataFrame({b: [mip_sol["shed"][(b,t)] for t in range(T)] for b in buses})
    E = np.array([mip_sol["e"][t] for t in range(T)])
    C = np.array([mip_sol["c"][t] for t in range(T)])
    D = np.array([mip_sol["d"][t] for t in range(T)])

    supply_net = Pg.sum(axis=1) + ren_df.sum(axis=1) + D - C
    demand_total = dem_df.sum(axis=1)
    plt.figure()
    plt.plot(demand_total, label="Demand (total)")
    plt.plot(supply_net, label="Supply (gen+ren+dis - ch)")
    plt.title("Supply vs Demand (MIP)")
    plt.xlabel("Time"); plt.ylabel("MW")
    plt.legend(); plt.tight_layout()

    plt.figure()
    plt.plot(E)
    plt.title("Storage Energy (SOC) — MIP")
    plt.xlabel("Time"); plt.ylabel("MWh")
    plt.tight_layout()

    # Line flow snapshot
    t_show = int(np.argmax(demand_total))
    flows = [mip_sol["f"][(ell,t_show)] for ell in range(len(line_df))]
    df_fl = line_df[["fr","to","Fmax"]].copy()
    df_fl["flow@peak"] = flows
    display(df_fl.head())
else:
    print("No MIP solution to summarize.")



## Part 2 — QUBO (Binary On/Off per Generator & Time) + Continuous Dispatch

We simplify to **system-wide** balance (ignore network constraints in QUBO for tractability):

- Binary \(y_{g,t}\): generator \(g\) online at time \(t\).  
- Penalty for balance: \(\big(\sum_g P^{\max}_g y_{g,t} + R_t - D_t\big)^2\), where \(R_t\) is renewable total and \(D_t\) total demand.  
- Ramp smoothness: \((y_{g,t} - y_{g,t-1})^2\).  
- Soft commitment count target per period to avoid over-commitment.

After solving the QUBO (via `neal`), we compute a **continuous dispatch** (LP) that respects generator bounds and **meets demand** using the committed set (still ignoring network lines in this dispatch step).


In [ ]:

from collections import defaultdict
import numpy as np, pandas as pd
import dimod, neal

G = len(gen_df)
Pmax = gen_df["pmax"].values
cost = gen_df["c"].values
Ramp = gen_df["ramp"].values
Dtot = dem_df.values.sum(axis=1)        # total demand per t
Rtot = ren_df.values.sum(axis=1)        # total renewables per t

def vkey(g,t): return g*T + t  # unique variable id

# Weights
A = float(5.0 * np.max(Pmax)**2)   # balance
B = 2.0                            # ramp smoothing
C = 0.5                            # soft commitment count
# cost weight: approximate cost per on-unit (max output * cost)
alpha_cost = 0.01

Q = defaultdict(float)

# (1) Balance per t: (sum_g Pmax_g y_gt + R_t - D_t)^2
for t in range(T):
    # Linear terms
    for g in range(G):
        Q[(vkey(g,t), vkey(g,t))] += A*(Pmax[g]**2 + 2*Pmax[g]*(Rtot[t]-Dtot[t]))
    # Pairwise across generators
    for g in range(G):
        for h in range(g+1, G):
            Q[(vkey(g,t), vkey(h,t))] += 2*A*Pmax[g]*Pmax[h]

# (2) Ramping: (y_gt - y_g,t-1)^2 = y+y -2yy
for g in range(G):
    for t in range(1, T):
        Q[(vkey(g,t), vkey(g,t))] += B
        Q[(vkey(g,t-1), vkey(g,t-1))] += B
        Q[(vkey(g,t-1), vkey(g,t))] += -2*B

# (3) Soft commitment count target per period
target_online = max(1, int(np.ceil(Dtot.max() / (Pmax.mean()+1e-9))))
for t in range(T):
    for g in range(G):
        Q[(vkey(g,t), vkey(g,t))] += C*(1 - 2*target_online)
    for g in range(G):
        for h in range(g+1, G):
            Q[(vkey(g,t), vkey(h,t))] += 2*C

# (4) Cost bias
for g in range(G):
    for t in range(T):
        Q[(vkey(g,t), vkey(g,t))] += alpha_cost * (cost[g]*Pmax[g])

# Solve QUBO
bqm = dimod.BinaryQuadraticModel.from_qubo(dict(Q))
sampleset = neal.SimulatedAnnealingSampler().sample(bqm, num_reads=2500)
best = sampleset.first

Y = np.zeros((G,T), dtype=int)
for g in range(G):
    for t in range(T):
        Y[g,t] = int(best.sample.get(vkey(g,t), 0))

print("Average committed units per period (QUBO):", Y.sum(axis=0).mean().round(2))


In [ ]:

# Continuous dispatch given QUBO commitment (system-level, no lines)
import numpy as np, pandas as pd

def dispatch_given_commitment(Y, gen_df, Dtot, Rtot):
    try:
        import pulp
    except Exception:
        return "NoPuLP", None
    G, T = Y.shape
    prob = pulp.LpProblem("DispatchGivenCommitment", pulp.LpMinimize)
    p = pulp.LpVariable.dicts("p", (range(G), range(T)), lowBound=0)
    # Objective
    c = gen_df["c"].values
    prob += pulp.lpSum(c[g]*p[g][t] for g in range(G) for t in range(T))
    # Bounds & meet total demand
    pmax = gen_df["pmax"].values
    for g in range(G):
        for t in range(T):
            prob += p[g][t] <= pmax[g] * Y[g,t]
    for t in range(T):
        prob += pulp.lpSum(p[g][t] for g in range(G)) + Rtot[t] == Dtot[t]
    # Solve
    _ = prob.solve(pulp.PULP_CBC_CMD(msg=False))
    status = pulp.LpStatus[prob.status]
    if status not in ("Optimal", "Feasible"):
        return status, None
    P = np.array([[float(p[g][t].value()) for t in range(T)] for g in range(G)])
    obj = float(pulp.value(prob.objective))
    return status, {"P": P, "obj": obj}

disp_status, disp = dispatch_given_commitment(Y, gen_df, Dtot, Rtot)
print("Dispatch LP status:", disp_status)
if disp:
    print("Dispatch cost (QUBO commitment):", round(disp["obj"],2))


In [ ]:

# Compare MIP vs QUBO+Dispatch and plot
import numpy as np, pandas as pd, matplotlib.pyplot as plt

if disp and mip_sol:
    Pg_qubo = pd.DataFrame(disp["P"].T, columns=gen_df["gen"])
    supply_qubo = Pg_qubo.sum(axis=1) + ren_df.sum(axis=1)
    demand_total = dem_df.sum(axis=1)

    plt.figure()
    plt.plot(demand_total, label="Demand")
    plt.plot(supply_qubo, label="Supply (QUBO+dispatch)")
    plt.title("Supply vs Demand — QUBO Commitment + LP Dispatch")
    plt.xlabel("Time"); plt.ylabel("MW")
    plt.legend(); plt.tight_layout()

    print("MIP total cost:", round(mip_sol["obj"],2))
    print("QUBO+Dispatch total cost:", round(disp['obj'],2))
elif disp:
    print("Only QUBO+dispatch available.")
elif mip_sol:
    print("Only MIP available.")
else:
    print("No solutions available.")



## Notes & Extensions

- The **MIP** uses a standard DC approximation and captures network constraints, storage dynamics, and ramping.  
- The **QUBO** is a simplified system-level **unit commitment** surrogate; you can extend it with per-generator min-up/down times, startup costs, and basic storage coupling.  
- To incorporate the **network** into the QUBO, consider time-indexed penalty terms for nodal balances and linearized flows on small grids (but variable counts grow quickly).  
- Try playing with generator costs, line limits, and storage parameters to see different balancing behaviors.
